In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import rasterio
import firedrake
from firedrake import Constant, inner, grad, dx
import pyadjoint
from firedrake_adjoint import Control, ReducedFunctional
import icepack

In the previous notebook, we created a mesh of the domain.

In [ ]:
mesh = firedrake.Mesh("sullivan-creek.msh")

To describe functions like the elevation field on the domain, we need to define the approximation space where they'll live.
Here we use piecewise quadratic basis functions in each triangle.

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2)

We can interpolate the DEM to this function space directly using a routine in icepack.

In [ ]:
with rasterio.open("elevation.tif", "r") as source:
    z = icepack.interpolate(source, Q)

In [ ]:
fig, axes = plt.subplots(subplot_kw={"projection": "3d"})
axes.view_init(azim=70)
firedrake.trisurf(z, axes=axes);

Our model is only well-posed when the slope of the elevation field is smaller than the critical slope $S_c$, which is usually taken to be about 1.25.
If we plot the slope of the elevation field, however, we found several patches where the slope exceeds the critical slope.

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, "DG", 1)
v = firedrake.project(grad(z), V)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(v, axes=axes)
contours = firedrake.tricontourf(v, levels=[0.0, 1.25], extend="max", axes=axes)
fig.colorbar(contours);

We could reduce the extent of the problem by re-drawing the domain boundary to exclude some of the regions of high slopes, but many of the problem areas are well inside the interior of our desired domain.
We can also mitigate this effect by applying a smoothing kernel, either to the input DEM or to the elevation field we interpolate to the unstructured mesh.
Applying a smoothing kernel everywhere can distort the elevation field even where nothing is wrong.

Instead of smoothing the whole elevation field, we'll treat this as a data assimilation problem.
We want to come up with an elevation field on the unstructured mesh that matches the DEM as close as possible overall.
We'll add the constraint that the slope can't be greater than the critical slope $S_c$.
This constraint is not smooth, so we'll have to do a bit of convex optimization to make things go.

First, we'll read in the DEM.

In [ ]:
with rasterio.open("elevation.tif", "r") as source:
    bounds = source.bounds
    transform = source.transform
    dem = source.read(indexes=1, masked=True)

Next, we need to find all the grid points of the DEM that lie inside the unstructured mesh.
The mesh has a `locate_cell` method that conveniently does this for us.
**TODO**: Find the min / max indices in both directions that bound the domain.

In [ ]:
shape = dem.shape
grid = itertools.product(range(shape[0]), range(shape[1]))

def inside(index):
    return mesh.locate_cell(transform * index) is not None

indices = np.array(list(filter(inside, grid)))
xs = np.array([transform * (i, j) for i, j in indices])

We can see by doing a scatter plot of the grid points on top of a small section of the triangular mesh that the DEM has a much higher data density than our finite element model will.
With that information in mind, we should expect that the elevation field we compute will not match all the grid points to within statistical noise because we're using far too few degrees of freedom.
It's also important to remember that the DEM is likely to capture microtopographical variability from sources like tree throw or even bioturbation that our mathematical model was never designed to simulate directly.
Trying to use a discretization at the same scale as the DEM might then be beside the point.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.set_xlim((351.6e3, 351.8e3))
axes.set_ylim((647.1e3, 647.3e3))
firedrake.triplot(mesh, axes=axes)
axes.scatter(xs[:, 0], xs[:, 1], 0.1, marker=".");

A vertex-only mesh represents a point cloud embedded into an unstructured triangular mesh.
We can create function spaces on the point cloud just like we can on the original mesh.
Here we'll create a function `z_obs` defined on the point cloud and manually pack its values with those of the DEM at the interior grid points.

In [ ]:
point_cloud = firedrake.VertexOnlyMesh(mesh, xs)
Q_obs = firedrake.FunctionSpace(point_cloud, "DG", 0)
z_obs = firedrake.Function(Q_obs)
z_obs.dat.data[:] = dem[indices[:, 1], indices[:, 0]]

The key enabling feature here is that (1) we can interpolate the elevation field $z$, which is defined everywhere in the domain of interest, to the point cloud, and (2) that this interpolation is differentiable, so we can use it in an optimization problem.

In [ ]:
Πz = firedrake.interpolate(z, Q_obs)

The metadata included with the DEM states that the LiDAR elevations are accurate to about 0.3 ft, so we'll take that as our measurement standard deviations.

In [ ]:
N = Constant(len(indices))
σ = Constant(0.3)
E = (Πz - z_obs)**2 / (2 * N * σ**2) * dx
firedrake.assemble(E)